<a href="https://colab.research.google.com/github/db030220/Plogging/blob/yolotrain/yolo_training%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Connect COLAB with Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Check your GPU compatibility with CUDA, as you can see COLAB GPU has CUDA 10.0 installed

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


# Load dataset and darknet folder to COLAB working space by unzip file "darknet.zip"

 * Use syntax !unzip + <path_to_darknet.zip_in_your_GDrive> to extract folder darknet. In my case, "darknet.zip" locates in My Drive/Colab Notebooks/yolo
 

In [ ]:
!unzip "/content/drive/MyDrive/2023finalproject/darknet.zip"

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: darknet/data/hand/puls_2_0_8.txt  
  inflating: darknet/data/hand/puls_2_0_8023.jpg  
  inflating: darknet/data/hand/puls_2_0_8023.txt  
  inflating: darknet/data/hand/puls_2_0_8032.jpg  
  inflating: darknet/data/hand/puls_2_0_8032.txt  
  inflating: darknet/data/hand/puls_2_0_8045.jpg  
  inflating: darknet/data/hand/puls_2_0_8045.txt  
  inflating: darknet/data/hand/puls_2_0_8085.jpg  
  inflating: darknet/data/hand/puls_2_0_8085.txt  
  inflating: darknet/data/hand/puls_2_0_8089.jpg  
  inflating: darknet/data/hand/puls_2_0_8089.txt  
  inflating: darknet/data/hand/puls_2_0_8111.jpg  
  inflating: darknet/data/hand/puls_2_0_8111.txt  
  inflating: darknet/data/hand/puls_2_0_8117.jpg  
  inflating: darknet/data/hand/puls_2_0_8117.txt  
  inflating: darknet/data/hand/puls_2_0_8123.jpg  
  inflating: darknet/data/hand/puls_2_0_8123.txt  
  inflating: darknet/data/hand/puls_2_0_8132.jpg  
  inflating: darknet/data/hand/puls_2_0_8132.txt 


#Compile darknet directory with below script:

In [ ]:
%cd /content/darknet
!make
!chmod +x ./darknet

/content/darknet
make: *** No targets specified and no makefile found.  Stop.
chmod: cannot access './darknet': No such file or directory


#Save weight during training in your Google Drive

This step is important since COLAB environment will be recycle after 12 hours and all files locate in its working space will be deleted. Here we defines a symbolic link to save the weight directly into our backup folder which we created in our GDrive before. In my case, my backup folder directory is My Drive/YOLOv3_weight/backup.

In [ ]:
!rm /content/darknet/backup -r
!ln -s /content/drive/'My Drive'/YOLO/backup /content/darknet

rm: cannot remove '/content/darknet/backup': No such file or directory


Install dos2unix to convert train.txt, val.txt, yolo.data, yolo.names, yolov3_custom_train.cfg to unix

In [ ]:
!sudo apt install dos2unix

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  dos2unix
0 upgraded, 1 newly installed, 0 to remove and 24 not upgraded.
Need to get 374 kB of archives.
After this operation, 1,342 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 dos2unix amd64 7.4.0-2 [374 kB]
Fetched 374 kB in 2s (245 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package dos2unix.
(Reading database ... 122518 files and directories current

In [ ]:
!dos2unix ./data/train.txt
!dos2unix ./data/val.txt
!dos2unix ./data/percent.data
!dos2unix ./data/percent.names
!dos2unix ./cfg/yolov3.cfg

dos2unix: ./data/train.txt: No such file or directory
dos2unix: Skipping ./data/train.txt, not a regular file.
dos2unix: ./data/val.txt: No such file or directory
dos2unix: Skipping ./data/val.txt, not a regular file.
dos2unix: converting file ./data/percent.data to Unix format...
dos2unix: converting file ./data/percent.names to Unix format...
dos2unix: converting file ./cfg/yolov3.cfg to Unix format...


In [ ]:
%cd /content/darknet
!./darknet detector train data/percent.data cfg/yolov3.cfg darknet53.conv.74

/content/darknet
/bin/bash: ./darknet: No such file or directory


In [ ]:
!./darknet detector test data/percent.data cfg/yolov3.cfg backup/yolov3_custom_train_1000.weights data/images/00001.jpg -out out

/bin/bash: ./darknet: No such file or directory
